In [7]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
from MCForecastTools import MCSimulation
import json

%matplotlib inline

In [8]:
?MCSimulation

Init signature:
MCSimulation(
    portfolio_data,
    weights='',
    num_simulation=1000,
    num_trading_days=252,
)
Docstring:     
A Python class for runnning Monte Carlo simulation on portfolio price data. 

...

Attributes
----------
portfolio_data : pandas.DataFrame
    portfolio dataframe
weights: list(float)
    portfolio investment breakdown
nSim: int
    number of samples in simulation
nTrading: int
    number of trading days to simulate
simulated_return : pandas.DataFrame
    Simulated data from Monte Carlo
confidence_interval : pandas.Series
    the 95% confidence intervals for simulated final cumulative returns
    
Init docstring:
Constructs all the necessary attributes for the MCSimulation object.

Parameters
----------
portfolio_data: pandas.DataFrame
    DataFrame containing stock price information from Alpaca API
weights: list(float)
    A list fractions representing percentage of total investment per stock. DEFAULT: Equal distribution
num_simulation: int
    Number 

In [9]:
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [10]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [11]:
today = pd.Timestamp("2020-06-01", tz="America/New_York").isoformat()


In [12]:
tickers = ["SPY", "AGG"]

In [13]:
timeframe = "1Day"

In [14]:
start_date = pd.Timestamp("1990-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()

In [51]:
# Get closing prices for SPY & AGG for past 31 years
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_portfolio.index.name='Date'
df_portfolio = df_portfolio.drop(columns=['open','high','low','volume','trade_count','vwap'])
df_portfolio

,close,symbol
Date,,
2015-12-01 05:00:00+00:00,108.82,AGG
2015-12-02 05:00:00+00:00,108.67,AGG
2015-12-03 05:00:00+00:00,107.89,AGG
2015-12-04 05:00:00+00:00,108.24,AGG
2015-12-07 05:00:00+00:00,108.40,AGG
...,...,...
2021-12-27 05:00:00+00:00,477.26,SPY
2021-12-28 05:00:00+00:00,476.87,SPY
2021-12-29 05:00:00+00:00,477.48,SPY


In [52]:
# Get closing prices for SPY & AGG for past 31 years

SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol', axis=1)
AGG = df_portfolio[df_portfolio['symbol']=='AGG'].drop('symbol', axis=1)

df_portfolio_year = pd.concat([SPY, AGG],axis=1, keys=['SPY','AGG'])

# set in ascending order (past to present)
df_portfolio_year = df_portfolio_year.sort_index()



df_portfolio_year.head()

,SPY,AGG
,close,close
Date,,
2015-12-01 05:00:00+00:00,210.68,108.82
2015-12-02 05:00:00+00:00,208.54,108.67
2015-12-03 05:00:00+00:00,205.58,107.89
2015-12-04 05:00:00+00:00,209.66,108.24
2015-12-07 05:00:00+00:00,208.27,108.40


In [53]:
daily_returns = df_portfolio_year.pct_change().dropna()
daily_returns.head()

,SPY,AGG
,close,close
Date,,
2015-12-02 05:00:00+00:00,-0.010158,-0.001378
2015-12-03 05:00:00+00:00,-0.014194,-0.007178
2015-12-04 05:00:00+00:00,0.019846,0.003244
2015-12-07 05:00:00+00:00,-0.006630,0.001478
2015-12-08 05:00:00+00:00,-0.006146,0.000369


In [54]:
# Create a simulation object
# This portfolio will have a 80/20 split between msft and aapl set in the weight parameter
# We set the number of simulations trials to be 100
# The period over which we will simulate is the number of trading days in a year times the number of years until the child reaches college.
# for this example, the child is 8 years old (meaning 10 years until college)
higher_risk_df = MCSimulation(
    portfolio_data=df_portfolio_year,
    weights=[0.80, 0.20],
    num_simulation=100,
    num_trading_days=252*10,
)

In [55]:
higher_risk_df.portfolio_data.head()

SPY                  AGG             
                            close daily_return   close daily_return
Date                                                               
2015-12-01 05:00:00+00:00  210.68          NaN  108.82          NaN
2015-12-02 05:00:00+00:00  208.54    -0.010158  108.67    -0.001378
2015-12-03 05:00:00+00:00  205.58    -0.014194  107.89    -0.007178
2015-12-04 05:00:00+00:00  209.66     0.019846  108.24     0.003244
2015-12-07 05:00:00+00:00  208.27    -0.006630  108.40     0.001478

In [56]:
higher_risk_df=(higher_risk_df[['close']])
display(higher_risk_df.head())

TypeError: 'MCSimulation' object is not subscriptable

In [21]:
# Cumulative Returns of higher risk portfolio
higher_risk_cumulative_returns = (1 + higher_risk_df).cumprod()
higher_risk_cumulative_returns.head()

TypeError: unsupported operand type(s) for +: 'int' and 'MCSimulation'

In [22]:
# Set weights for low risk portfolio (20% SPY, 80% AGG)
low_risk_df = MCSimulation(
    portfolio_data=df_portfolio_year,
    weights=[0.20, 0.80],
    num_simulation=100,
    num_trading_days=252*10,
)

In [23]:
low_risk_df.portfolio_data.head()

SPY                                         \
                             open      high       low   close     volume   
timestamp                                                                  
2015-12-01 05:00:00+00:00  209.42  210.8200  209.1100  210.68   97858418   
2015-12-02 05:00:00+00:00  210.60  211.0000  208.2300  208.54  108069059   
2015-12-03 05:00:00+00:00  208.90  209.1500  204.7511  205.58  166224154   
2015-12-04 05:00:00+00:00  206.10  209.9700  205.9300  209.66  192878747   
2015-12-07 05:00:00+00:00  209.20  209.7295  207.2000  208.27  102027111   

                                                                   AGG  \
                          trade_count        vwap daily_return    open   
timestamp                                                                
2015-12-01 05:00:00+00:00      337780  209.921410          NaN  108.54   
2015-12-02 05:00:00+00:00      367013  209.563055    -0.010158  108.73   
2015-12-03 05:00:00+00:00      546768  206.878936    -0.014194  108.41   
2015-12-04 05:00:00+00:00      556731  208.178631     0.019846  108.05   
2015-12-07 05:00:00+00:00      374705  208.276128    -0.006630  108.30   

                                                                          \
                               high     low   close   volume trade_count   
timestamp                                                                  
2015-12-01 05:00:00+00:00  108.8400  108.43  108.82  4259986       15562   
2015-12-02 05:00:00+00:00  108.7799  108.61  108.67  2462787       11581   
2015-12-03 05:00:00+00:00  108.4100  107.81  107.89  4634020       16801   
2015-12-04 05:00:00+00:00  108.3000  108.00  108.24  2182057        9796   
2015-12-07 05:00:00+00:00  108.5800  108.23  108.40  2143773        9104   

                                                    
                                 vwap daily_return  
timestamp                                           
2015-12-01 05:00:00+00:00  108.754152          NaN  
2015-12-02 05:00:00+00:00  108.684692    -0.001378  
2015-12-03 05:00:00+00:00  108.040315    -0.007178  
2015-12-04 05:00:00+00:00  108.192232     0.003244  
2015-12-07 05:00:00+00:00  108.460067     0.001478

In [32]:
# Cumulative Returns of low risk portfolio
low_risk_cumulative_returns = (1 + low_risk_df).cumprod()-1
low_risk_cumulative_returns.head()

TypeError: unsupported operand type(s) for +: 'int' and 'MCSimulation'

In [33]:
agg_daily_return = AGG.pct_change()
agg_daily_return

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-02 05:00:00+00:00,0.001751,-0.000552,0.001660,-0.001378,-0.421879,-0.255815,-0.000639
2015-12-03 05:00:00+00:00,-0.002943,-0.003400,-0.007366,-0.007178,0.881616,0.450738,-0.005929
2015-12-04 05:00:00+00:00,-0.003321,-0.001015,0.001762,0.003244,-0.529122,-0.416939,0.001406
2015-12-07 05:00:00+00:00,0.002314,0.002585,0.002130,0.001478,-0.017545,-0.070641,0.002476
...,...,...,...,...,...,...,...
2021-12-27 05:00:00+00:00,-0.001225,-0.000087,0.000526,0.000525,-0.012771,-0.098702,0.000701
2021-12-28 05:00:00+00:00,0.002453,0.001312,0.000350,-0.000438,-0.033580,0.056784,0.000453
2021-12-29 05:00:00+00:00,-0.003845,-0.003495,-0.003153,-0.002977,0.128607,0.111431,-0.003318


In [34]:
agg_cumulative_return = (1 + agg_daily_return).cumprod() - 1
agg_cumulative_return

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-02 05:00:00+00:00,0.001751,-0.000552,0.001660,-0.001378,-0.421879,-0.255815,-0.000639
2015-12-03 05:00:00+00:00,-0.001198,-0.003951,-0.005718,-0.008546,0.087802,0.079617,-0.006564
2015-12-04 05:00:00+00:00,-0.004514,-0.004961,-0.003966,-0.005330,-0.487778,-0.370518,-0.005167
2015-12-07 05:00:00+00:00,-0.002211,-0.002389,-0.001845,-0.003860,-0.496765,-0.414985,-0.002704
...,...,...,...,...,...,...,...
2021-12-27 05:00:00+00:00,0.051686,0.050165,0.052568,0.049991,0.276140,-0.049415,0.050487
2021-12-28 05:00:00+00:00,0.054266,0.051544,0.052937,0.049531,0.233287,0.004562,0.050963
2021-12-29 05:00:00+00:00,0.050212,0.047868,0.049617,0.046407,0.391896,0.116502,0.047476
